# Imports and Downloads  
### NLTK library is used as the main resource for the NLP algorithm 
### The following inputs are used to:  
    1. 'twitter_samples': create the model which is based out of tweets so samples may be downloaded for the dataset  
    2. 'punkt': module which is pre-trained to tokenize words and sentences  
    3. 'wordnet': lexical database for the English language  
    4. 'averaged_perceptron_tagger': resource to determine the context of a word in a sentence  
    5. 'stopwords': common stopwords in English language

In [3]:
# import NLTK library which is the Natural Language Toolkit adn other necessary sub-librairies

import nltk
from nltk import FreqDist
from nltk.corpus import twitter_samples
from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import classify
from nltk import NaiveBayesClassifier

import re, string
import random

In [4]:
nltk.download('twitter_samples')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package twitter_samples to
[nltk_data]     /Users/alexandreberkovic/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/alexandreberkovic/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/alexandreberkovic/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/alexandreberkovic/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/alexandreberkovic/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Pre-processing and data cleaning

In [5]:
# importing the positive and negative tweets to create the dataset (5k points each)
# the text variable is a dataset of 20k points with no sentiments
positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')
text = twitter_samples.strings('tweets.20150430-223406.json')

In [6]:
# pos_tag is a tagging algorithm which assesses the relative position of a word in a sentence
tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
print(pos_tag(tweet_tokens[0]))

[('#FollowFriday', 'JJ'), ('@France_Inte', 'NNP'), ('@PKuchly57', 'NNP'), ('@Milipol_Paris', 'NNP'), ('for', 'IN'), ('being', 'VBG'), ('top', 'JJ'), ('engaged', 'VBN'), ('members', 'NNS'), ('in', 'IN'), ('my', 'PRP$'), ('community', 'NN'), ('this', 'DT'), ('week', 'NN'), (':)', 'NN')]


In [7]:
def lemmatize_sentence(tokens):
    '''
    Lemmatization normalises a word with the context of vocabulary
    and mophological anlysis of words in text. It analyses the structure of the word and its context to normalise it.
    '''
    lemmatizer = WordNetLemmatizer()
    lemmatized_sentence = []
    for word, tag in pos_tag(tokens):
        if tag.startswith('NN'):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        lemmatized_sentence.append(lemmatizer.lemmatize(word, pos))
    return lemmatized_sentence

print(lemmatize_sentence(tweet_tokens[0]))

['#FollowFriday', '@France_Inte', '@PKuchly57', '@Milipol_Paris', 'for', 'be', 'top', 'engage', 'member', 'in', 'my', 'community', 'this', 'week', ':)']


In [8]:
def remove_noise(tweet_tokens, stop_words = ()):
    '''
    Removes noise from the dataset such as hyperlinks, punctuation, special characters and incroporatrs the normalisation and lemmatization mentioned previously.
    '''
    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

In [9]:
stop_words = stopwords.words('english')
print(remove_noise(tweet_tokens[0], stop_words))

['#followfriday', 'top', 'engage', 'member', 'community', 'week', ':)']


In [80]:
# this tokenizes the data by sequencing the characters in the text as individual units and enables to process natural language
positive_tokens = twitter_samples.tokenized('positive_tweets.json')
negative_tokens = twitter_samples.tokenized('negative_tweets.json')

positive_cleaned_tokens_list = []
negative_cleaned_tokens_list = []

for tokens in positive_tokens:
    positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

for tokens in negative_tokens:
    negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

In [81]:
print(positive_tokens[500])
print(positive_cleaned_tokens_list[500])

['Dang', 'that', 'is', 'some', 'rad', '@AbzuGame', '#fanart', '!', ':D', 'https://t.co/bI8k8tb9ht']
['dang', 'rad', '#fanart', ':d']


In [82]:
def get_all_words(cleaned_tokens_list):
    '''
    generator function that takes a list of corpuses as arguments to provide a list of tokens in all of the corpuses.
    '''
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token

all_pos_words = get_all_words(positive_cleaned_tokens_list)

In [83]:
# enables to display the most common words
freq_dist_pos = FreqDist(all_pos_words)
print(freq_dist_pos.most_common(10))

[(':)', 3691), (':-)', 701), (':d', 658), ('thanks', 388), ('follow', 357), ('love', 333), ('...', 290), ('good', 283), ('get', 263), ('thank', 253)]


In [84]:
def get_tweets_for_model(cleaned_tokens_list):
    '''
    generator function that changes the format of the cleaned data to a Python dictionnary so that it may be fed to the model
    '''
    for tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)

positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)

# Dataset Creation

In [85]:
# creates a positive dataset
for token in positive_tokens_for_model:
    positive_df.append((token, "Positive"))

In [86]:
# creates a negative dataset
for token in negative_tokens_for_model:
    negative_df.append((token, "Negative"))

In [89]:
# combines the datasets, shuffles them and splits them into a training set and a test set
df = list(positive_df) + negative_df
random.shuffle(df)

train_df = df[:7000]
test_df = df[7000:]

# Model training

In [90]:
# we use a simple Naive Bayes Classifier to buid the model as it displays high accuracies
classifier = NaiveBayesClassifier.train(train_df)
print("Accuracy is:", classify.accuracy(classifier, test_df))
print(classifier.show_most_informative_features(10))

Accuracy is: 0.49838461538461537
Most Informative Features
           #FollowFriday = True           Negati : Positi =      1.0 : 1.0
                      :) = True           Negati : Positi =      1.0 : 1.0
            @France_Inte = True           Negati : Positi =      1.0 : 1.0
          @Milipol_Paris = True           Negati : Positi =      1.0 : 1.0
              @PKuchly57 = True           Negati : Positi =      1.0 : 1.0
                   being = True           Negati : Positi =      1.0 : 1.0
               community = True           Negati : Positi =      1.0 : 1.0
                 engaged = True           Negati : Positi =      1.0 : 1.0
                     for = True           Negati : Positi =      1.0 : 1.0
                      in = True           Negati : Positi =      1.0 : 1.0
None


In [91]:
# create a custom token to test the model
custom_sentence = "what a beautiful day i feel great"
custom_tokens = remove_noise(word_tokenize(custom_sentence))

print(classifier.classify(dict([token, True] for token in custom_tokens)))

Negative


# Flagging
After conversations with mental health care nurses, we quickly understood that key sentences needed to be flagged for immediate action to be taken by carers or relatives. Below is that implementation.

In [92]:
sentences = ['i feel depressed', 'i cannot do it anymore', '...']
text = "i feel depressed"

In [93]:
def warning(text, sentences):
    '''
    Flags sentences which should directly warn carers or relatives 
    as they display extreme swings of emotion which can be harmful
    '''
    for sentence in sentences:
        if sentence == text:
            return 'WARNING'
            break
        else:
            pass

In [94]:
warning(text, sentences)

'WARNING'